In [3]:
# In this notebook, we consider a problem from the paper "Quantum advantage with shallow circuits" and build a quantum circuit, which solves it in Cirq.
try:
    import cirq
except ImportError:
    print("installing cirq..")
    

In [4]:
# Introduction
# It is well known that some problems can be solved on the qunatum compiter exponentionally faster
# than on classical one in terms of computation time. However, there is more subtle way in which quantum computers are more
# powerful. There is a problem, which can be solved by quantum circuit of constant depth, but can't be solved by classical circuit of constant depth.
# In this notebook we will consider this problem.

In [ ]:
# Structure of this notebook

In [5]:
import numpy as np
import cirq

In [8]:
class HiddenLinearFunctionProblem:
    """Instance of Hidden Linear Function problem.
    
    The problem is defined by matrix A and vector b, which are
    the coefficients of quadratic form, in which linear function
    is "hidden".
    """
    def __init__(self, A, b):
        self.n = A.shape[0]
        assert A.shape == (self.n, self.n)
        assert b.shape == (self.n, )
        for i in range(self.n):
            for j in range(i+1):
                assert A[i][j] == 0, 'A[i][j] can be 1 only if i>i) % 2 for i in range(self.n)]) for m in range(2**self.n)]'

        def vector_in_L(x):
            for y in all_vectors:
                if self.q( (x + y)%2 ) != (self.q(x) + self.q(y))%4:
                    return False
            return True

        # L is subspace to which we restrict domain of quadratic form.
        # Corresponds to `L_q` in the problem definition.
        self.L = [x for x in all_vectors if vector_in_L(x)]
        
        # All vectors `z` which are solutions to the problem.
        self.all_zs = [z for z in all_vectors if self.is_z(z)]
    
    def is_z(self, z):
        """Checks by definition, whether given vector `z` is solution to this problem."""
        assert z.shape == (self.n, )
        assert self.L is not None
        for x in self.L:
            if self.q(x) != 2 * ((z @ x) % 2):
                return False
        return True


In [9]:
# for testing, to generate an instance of a problem. Therefore generate random A and b.\

def random_problem(n, seed=None):
    "Generate instances of the problem with given n"

    "Args:"
    "n: dimension of the problem."
    if seed is not None:
        np.random.seed(seed)
    A = np.random.randint(0, 2, size=(n,n))
    for i in range(n):
        for j in range(i+1):
            A[i][j] = 0
        b = np.random.randint(0, 2, size=n)
        problem = HiddenLinearFunctionProblem(A, b)
        return problem
    
def find_interesting_problem(n, min_L_size):
    """ Generates interesting instance of the problem"""

    for _ in range(1000):
        problem = random_problem(n)
        problem.bruteforce_solve()
        if len(problem.L) >= min_L_size and not np.max(problem.A) == 0:
            return problem
        return None
    
    problem = find_interesting_problem(10, 4)
    print("Size of the subspace L: %d" % len(problem.L))
    print("Number of soulutions: %d" % len(problem.all_zs))



In [12]:
A = np.array([[0, 1, 1, 0, 0, 1, 0, 0, 1, 1],
              [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
              [0, 0, 0, 0, 0, 0, 1, 1, 0, 1],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
              [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
b = np.array([0, 0, 0, 0, 1, 1, 1, 0, 0, 1])
problem_10_64 = HiddenLinearFunctionProblem(A, b)
problem_10_64.bruteforce_solve()
print("Size of subspace L: %d" % len(problem_10_64.L))
print("Number of solutions: %d" % len(problem_10_64.all_zs))


NameError: name 'all_vectors' is not defined

In [13]:
def edge_coloring(A):
    A = np.copy(A)
    n = A.shape[0]
    ans = []
    while np.max(A) != 0:
        edges_group = []
        used = np.zeros(n, dtype=bool)
        for i in range(n):
            for j in range(n):
                if A[i][j] == 1 and not used[i] and not used[j]:
                    edges_group.append((i, j))
                    A[i][j] = 0
                    used[i] = used[j] = True
            ans.append(edges_group)
        return ans

In [ ]:
def generate_circuit_for_problem(problem):
    """ Generates cirq.Circuit which solves instance of Hidden Linear Function problem"""
    qubits = cirq.LineQubit.range(problem.n)
    circuit = cirq.Circuit()

    # Hadamard gates at the beginning (creating equal superposition of all states).
    circuit += cirq.Moment([cirq.H(q) for q in qubits])

    # Controlled-z gates encoding the matrix A.
    for layer in edge_coloring(problem.A):
        for i, j in layer:
            circuit += cirq.CZ(qubits[i], qubits[j])

    # S gates encoding the vector b.
    circuit += cirq.Moment([cirq.S.on(qubits[i]) for i in range(problem.n) if problem.b[i] == 1])

    # Hadamard gates at the end.
    circuit += cirq.Moment([cirq.measure(qubits[i], key=str(i)) for i in range(problem.n)]) 

    return circuit

def solve_problem(problem, print_circuit=False):
    """ Solve instance of Hidden Linear Function problem.
     
      Builds quantum circuit for given problem and simulates it with the Clifford
       simulator.
    
     Returns measurement result as binary vector, which is guaranteed to a solution to given problem.
       """
    circuit = generate_circuit_for_problem(problem)

    if print_circuit:
        print(circuit)
    sim = cirq.CliffordSimulator()
    result = sim.simulate(circuit)
    z = np.array([result.measurements[str(i)][0] for i in range(problem.n)])
    return z

solve_problem(problem_10_64, print_circuit=True)


In [ ]:
# Testing 
# To solve it with a quantum circuit 100 times and each time check that measurement result is indeed an answer to the problem.
def test_problem(problem):
    problem.bruteforce_solve()
    tries = 100
    for _ in range(tries):
        z = solve_problem(problem)
        assert problem.is_z(z)

test_problem(problem_10_64)
print('OK')

In [ ]:
# Repeat that for 10 other problems with n = 8 and chance of random guessing at most one 4th.
for _ in range(10):
    test_problem(find_interesting_problem(8, 4))
print('OK')

In [ ]:
%%time
tries = 200
problem = random_problem(tries, seed=0)
solve_problem(problem, print_circuit=False)